##### Copyright 2019 The TensorFlow Authors.

In [ ]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.


# Создание линейной модели с помощью эстиматоров

<table class="tfo-notebook-buttons" align="left">
  <td>
    <a target="_blank" href="https://www.tensorflow.org/tutorials/estimator/linear"><img src="https://www.tensorflow.org/images/tf_logo_32px.png" />Смотреть на TensorFlow.org</a>
  </td>
  <td>
    <a target="_blank" href="https://colab.research.google.com/github/tensorflow/docs-l10n/blob/master/site/ru/tutorials/estimator/linear.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png" />Запустить в Google Colab</a>
  </td>
  <td>
    <a target="_blank" href="https://github.com/tensorflow/docs-l10n/blob/master/site/ru/tutorials/estimator/linear.ipynb"><img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png" />Смотреть исходные файлы на GitHub</a>
  </td>
  <td>
    <a href="https://storage.googleapis.com/tensorflow_docs/docs-l10n/site/ru/tutorials/estimator/linear.ipynb"><img src="https://www.tensorflow.org/images/download_logo_32px.png" />Скачать ноутбук</a>
  </td>
</table>

Note: Вся информация в этом разделе переведена с помощью русскоговорящего Tensorflow сообщества на общественных началах. Поскольку этот перевод не является официальным, мы не гарантируем что он на 100% аккуратен и соответствует [официальной документации на английском языке](https://www.tensorflow.org/?hl=en). Если у вас есть предложение как исправить этот перевод, мы будем очень рады увидеть pull request в [tensorflow/docs](https://github.com/tensorflow/docs) репозиторий GitHub. Если вы хотите помочь сделать документацию по Tensorflow лучше (сделать сам перевод или проверить перевод подготовленный кем-то другим), напишите нам на [docs-ru@tensorflow.org list](https://groups.google.com/a/tensorflow.org/forum/#!forum/docs-ru).

## Введение

Это полное пошаговое руководство обучения модели логистической регрессии с использованием `tf.estimator` API. Эта модель часто используется в качестве основы для других, более сложных алгоритмов.


## Установка

In [ ]:
!pip install sklearn


In [ ]:
import os
import sys

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from IPython.display import clear_output
from six.moves import urllib

## Загрузка датасета Титаник

Вы будете использовать набор данных "Титаник", цель которого(довольно болезненная) - предсказать выживаемость пассажиров с учетом таких характеристик, как пол, возраст, класс и т.д.

In [ ]:
import tensorflow.compat.v2.feature_column as fc

import tensorflow as tf

In [ ]:
# Загружаем датасет
dftrain = pd.read_csv('https://storage.googleapis.com/tf-datasets/titanic/train.csv')
dfeval = pd.read_csv('https://storage.googleapis.com/tf-datasets/titanic/eval.csv')
y_train = dftrain.pop('survived')
y_eval = dfeval.pop('survived')

## Изучение данных

Датасет состоит из следующих признаков:

In [ ]:
dftrain.head()

In [ ]:
dftrain.describe()

В обучающей и оценочной выборках 627 и 264 примера соответственно.

In [ ]:
dftrain.shape[0], dfeval.shape[0]

Большинство пассажиров в возрасте от 20 до 30 лет.

In [ ]:
dftrain.age.hist(bins=20)

Пассажиров-мужчин примерно в два раза больше, чем пассажиров-женщин.

In [ ]:
dftrain.sex.value_counts().plot(kind='barh')

Большинство пассажиров находились в «третьем» классе.

In [ ]:
dftrain['class'].value_counts().plot(kind='barh')

У женщин намного больше шансов выжить, чем у мужчин. Это явно будет прогностическим признаком модели.

In [ ]:
pd.concat([dftrain, y_train], axis=1).groupby('sex').survived.mean().plot(kind='barh').set_xlabel('% survive')

## Разработка признаков для модели
Оценщики используют систему под названием [столбцы признаков](https://www.tensorflow.org/guide/feature_columns) для описания того, как модель должна интерпретировать кажды из исходных входных признаков. Оценщик ожидает вектор числовых входных данных, а *столбцы признаков* описывают, как модель должна преобразовывать каждый признак.

Выбор и создание правильного набора столбцов признаков - ключ к изучению эффективной модели. Столбец признаков может быть либо одним из исходных признаков(*столбец базовых признаков*), либо любым новым столбцом, созданными с использованием преобразований, определенных для одного или нескольких базовых столбцов(*столбцы производных признаков*).

Линейный эстиматор использует как числовые, так и категориальные признаки. Столбцы признаков работают со всеми оценщиками TensorFlow, и их цель - определить признаки, используемые для моделирования. Кроме того, они предоставляют некоторые возможности разработки признаков, такие как one-hot-encoding, нормализация и разделение на сегменты.

### Столбцы базовых признаков

In [ ]:
CATEGORICAL_COLUMNS = ['sex', 'n_siblings_spouses', 'parch', 'class', 'deck',
                       'embark_town', 'alone']
NUMERIC_COLUMNS = ['age', 'fare']

feature_columns = []
for feature_name in CATEGORICAL_COLUMNS:
  vocabulary = dftrain[feature_name].unique()
  feature_columns.append(tf.feature_column.categorical_column_with_vocabulary_list(feature_name, vocabulary))

for feature_name in NUMERIC_COLUMNS:
  feature_columns.append(tf.feature_column.numeric_column(feature_name, dtype=tf.float32))

`input_function` определяет, каким образом данные преобразуются в `tf.data.Dataset`, который подает данные во входной конвейер в потоковом режиме. `tf.data.Dataset` может принимать несколько источников, таких как pandas dataframe, файл в формате csv и многое другое.

In [ ]:
def make_input_fn(data_df, label_df, num_epochs=10, shuffle=True, batch_size=32):
  def input_function():
    ds = tf.data.Dataset.from_tensor_slices((dict(data_df), label_df))
    if shuffle:
      ds = ds.shuffle(1000)
    ds = ds.batch(batch_size).repeat(num_epochs)
    return ds
  return input_function

train_input_fn = make_input_fn(dftrain, y_train)
eval_input_fn = make_input_fn(dfeval, y_eval, num_epochs=1, shuffle=False)

Вы можете проверить датасет:

In [ ]:
ds = make_input_fn(dftrain, y_train, batch_size=10)()
for feature_batch, label_batch in ds.take(1):
  print('Some feature keys:', list(feature_batch.keys()))
  print()
  print('A batch of class:', feature_batch['class'].numpy())
  print()
  print('A batch of Labels:', label_batch.numpy())

Вы также можете проверить результат столбца с конкретным признаком, используя слой `tf.keras.layers.DenseFeatures`:

In [ ]:
age_column = feature_columns[7]
tf.keras.layers.DenseFeatures([age_column])(feature_batch).numpy()

`DenseFeatures` принимает только полносвязные тензоры, чтобы проверить категориальный столбец, вам нужно сначала преобразовать его в столбец-индикатор:

In [ ]:
gender_column = feature_columns[0]
tf.keras.layers.DenseFeatures([tf.feature_column.indicator_column(gender_column)])(feature_batch).numpy()

После добавления всех базовых признаков к модели давайте обучим модель. Обучение модели - это всего лишь одна команда с использованием API `tf.estimator`:

In [ ]:
linear_est = tf.estimator.LinearClassifier(feature_columns=feature_columns)
linear_est.train(train_input_fn)
result = linear_est.evaluate(eval_input_fn)

clear_output()
print(result)

### Столбцы производных признаков

Сейчас вы достигли точности 75%. Использование каждого столбца базовых характеристик отдельно может оказаться недостаточным для описания данных. Например, соотношение пола и метки может быть разным для мужчин и женщин. Таким образом, если вы изучите только один вес модели для  `gender="Male"` и `gender="Female"`, вы не сможете охватить каждую комбинацию возраст-пол (например, различие между `gender="Male"` и `age = "30"` и `gender="Male"`и `age ="40"`).

Чтобы узнать о различиях между разными комбинациями признаков, вы можете добавить в модель *столбцы перекрестных признаков*(вы также можете разбить столбец `age` на части перед перекрестным сравнением столбцов):

In [ ]:
age_x_gender = tf.feature_column.crossed_column(['age', 'sex'], hash_bucket_size=100)

После добавления комбинирования признаков в модель давайте снова обучим модель:

In [ ]:
derived_feature_columns = [age_x_gender]
linear_est = tf.estimator.LinearClassifier(feature_columns=feature_columns+derived_feature_columns)
linear_est.train(train_input_fn)
result = linear_est.evaluate(eval_input_fn)

clear_output()
print(result)

Теперь точность составляет 77,6%, что немного лучше, чем при обучении только базовым признакам. Вы можете попробовать использовать больше функций и преобразований, чтобы увидеть, сможете ли вы добиться большего!

Теперь вы можете обучить модель, чтобы делать прогнозы относительно пассажира на основе оценочного датасета. Модели TensorFlow оптимизированы, чтобы делать прогнозы сразу для пакета или для коллекции. Ранее мы указали в `eval_input_fn` использовать весь датасет целиком.

In [ ]:
pred_dicts = list(linear_est.predict(eval_input_fn))
probs = pd.Series([pred['probabilities'][1] for pred in pred_dicts])

probs.plot(kind='hist', bins=20, title='predicted probabilities')

И наконец, вы можете посмотреть на ROC-кривую, позволяющий оценить качество нашей классификации.

In [ ]:
from sklearn.metrics import roc_curve
from matplotlib import pyplot as plt

fpr, tpr, _ = roc_curve(y_eval, probs)
plt.plot(fpr, tpr)
plt.title('ROC curve')
plt.xlabel('false positive rate')
plt.ylabel('true positive rate')
plt.xlim(0,)
plt.ylim(0,)